In [4]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BasicEx').getOrCreate()

In [5]:
from pyspark.ml import pipeline

In [6]:
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier,DecisionTreeClassifier#can be used for regression also

In [7]:
data= spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [8]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [12]:
rf_classifer = RandomForestClassifier(numTrees=100)
gbt_classifer = GBTClassifier()
dt_classifer = DecisionTreeClassifier()

In [11]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [13]:
rf_model = rf_classifer.fit(train_data)

In [28]:
rf_model.featureImportances

SparseVector(692, {158: 0.0004, 181: 0.0002, 184: 0.0013, 208: 0.0001, 214: 0.0021, 217: 0.0013, 243: 0.0011, 244: 0.0093, 262: 0.0069, 263: 0.0017, 268: 0.0014, 271: 0.0008, 272: 0.0162, 273: 0.0016, 274: 0.0005, 290: 0.0095, 291: 0.0079, 293: 0.0013, 298: 0.0002, 300: 0.0235, 302: 0.0013, 315: 0.0024, 317: 0.0068, 320: 0.0012, 321: 0.0004, 323: 0.0082, 326: 0.0005, 328: 0.0281, 329: 0.0006, 330: 0.0168, 332: 0.0011, 341: 0.0019, 342: 0.0025, 345: 0.0211, 346: 0.002, 347: 0.0006, 348: 0.0012, 349: 0.0003, 350: 0.0246, 351: 0.0158, 352: 0.0007, 355: 0.0025, 356: 0.0007, 357: 0.0085, 358: 0.0006, 372: 0.0006, 373: 0.0149, 374: 0.007, 377: 0.0147, 378: 0.038, 379: 0.0421, 380: 0.0155, 384: 0.016, 386: 0.0082, 387: 0.0042, 388: 0.0012, 397: 0.0005, 398: 0.0004, 399: 0.0004, 400: 0.0126, 401: 0.0005, 403: 0.0006, 405: 0.0283, 406: 0.0242, 407: 0.0188, 410: 0.0021, 411: 0.0014, 412: 0.0102, 413: 0.0175, 425: 0.0013, 426: 0.001, 429: 0.0107, 430: 0.0006, 431: 0.0005, 433: 0.0388, 434: 0.0442

In [14]:
gbt_model = gbt_classifer.fit(train_data)

In [15]:
dt_model = dt_classifer.fit(train_data)

In [16]:
rf_model.numFeatures

692

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [24]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [17]:
rf_predict = rf_model.transform(test_data)

In [20]:
rf_predict.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [97.0,3.0]|[0.97,0.03]|       0.0|
|  0.0|(692,[98,99,100,1...|  [100.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|  [100.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|  [100.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|  [84.0,16.0]|[0.84,0.16]|       0.0|
|  0.0|(692,[126,127,128...|   [96.0,4.0]|[0.96,0.04]|       0.0|
|  0.0|(692,[126,127,128...|   [99.0,1.0]|[0.99,0.01]|       0.0|
|  0.0|(692,[126,127,128...|   [97.0,3.0]|[0.97,0.03]|       0.0|
|  0.0|(692,[127,128,129...|   [97.0,3.0]|[0.97,0.03]|       0.0|
|  0.0|(692,[150,151,152...|   [94.0,6.0]|[0.94,0.06]|       0.0|
|  0.0|(692,[152,153,154...|   [94.0,6.0]|[0.94,0.06]|       0.0|
|  0.0|(692,[153,154,155...|   [99.0,1.0]|[0.99,0.01]|       0.0|
|  0.0|(69

In [25]:
acc_eval.evaluate(rf_predict)

1.0

In [18]:
gbt_predict = gbt_model.transform(test_data)

In [21]:
gbt_predict.show()

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.0|(692,[95,96,97,12...|       0.0|
|  0.0|(692,[98,99,100,1...|       1.0|
|  0.0|(692,[123,124,125...|       0.0|
|  0.0|(692,[124,125,126...|       0.0|
|  0.0|(692,[124,125,126...|       0.0|
|  0.0|(692,[126,127,128...|       0.0|
|  0.0|(692,[126,127,128...|       0.0|
|  0.0|(692,[126,127,128...|       0.0|
|  0.0|(692,[127,128,129...|       0.0|
|  0.0|(692,[150,151,152...|       0.0|
|  0.0|(692,[152,153,154...|       0.0|
|  0.0|(692,[153,154,155...|       0.0|
|  0.0|(692,[153,154,155...|       0.0|
|  0.0|(692,[155,156,180...|       0.0|
|  1.0|(692,[123,124,125...|       1.0|
|  1.0|(692,[123,124,125...|       1.0|
|  1.0|(692,[123,124,125...|       1.0|
|  1.0|(692,[124,125,126...|       1.0|
|  1.0|(692,[124,125,126...|       1.0|
|  1.0|(692,[126,127,128...|       1.0|
+-----+--------------------+----------+
only showing top 20 rows



In [26]:
acc_eval.evaluate(gbt_predict)

0.9333333333333333

In [19]:
dt_predict = dt_model.transform(test_data)

In [22]:
dt_predict.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[98,99,100,1...|    [0.0,1.0]|  [0.0,1.0]|       1.0|
|  0.0|(692,[123,124,125...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[150,151,152...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(69

In [27]:
acc_eval.evaluate(dt_predict)

0.9333333333333333